# Plotly Dash with Dash Bootstrap: Interactive Data Visualization with Jupyter Notebooks

## Introduction

[Plotly Dash](https://plotly.com/dash/) is a powerful Python library for building web-based dashboards and applications. In this presentation, we will explore the capabilities of Plotly Dash in Jupyter Notebooks and how it can be used to create interactive data visualizations. Additionally, we will discuss the integration of [Dash Bootstrap](https://dash-bootstrap-components.opensource.faculty.ai/), a popular library that provides additional styling options and components to enhance the visual appeal of Dash applications.

## Overview of Plotly Dash

- Plotly Dash is an open-source Python framework for building web-based analytics applications.
- It is built on top of Plotly, a popular library for creating interactive visualizations.
- Dash provides a high-level API that simplifies the process of building interactive dashboards and web applications.

## Advantages of Plotly Dash in Jupyter Notebooks

1. Seamless Integration
   - Plotly Dash seamlessly integrates with Jupyter Notebooks, enabling you to create interactive visualizations within the familiar Jupyter environment.
2. Interactive Dashboards
   - With Dash, you can easily create interactive dashboards that allow users to explore and interact with data dynamically.
3. Pythonic Development
   - Plotly Dash is written in Python and follows Pythonic principles, making it accessible to Python developers of all levels.
4. Rich Component Library
   - Dash provides a wide range of pre-built components, such as sliders, dropdowns, and graphs, that can be easily customized to create visually appealing and interactive dashboards.

In [1]:
# example dashboard
import IPython

src = "https://shapash-demo.ossbymaif.fr/"
width = 1920
height = 1080
zoom_level = 0.5  # Adjust this value to scale the website

style = f"transform: scale({zoom_level}); transform-origin: top left;"

site = (
    f'<iframe src="{src}" width="{width}" height="{height}" style="{style}"></iframe>'
)
IPython.display.HTML(site)

/home/stefan/private/HSKA/plotly_presentation/venv/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Required packages
* `pandas` for data exploration
* `dash` the basic dash library
* `jupyter-dash` extention for getting dash working in a jupyter-notebook
* `dash-bootstrap-components` update look and feel of dash UI elements, and add more UI elements

### Install packages
```bash
# create new venv
python -m venv

# activate venv
venv/bin/activate

# install packages
pip install dash jupyter-dash pandas dash-bootstrap-components
```

## Example: Creating a Dash Dashboard

In [2]:
import pandas as pd

In [3]:
# simplified plotly library for genreating plots
import plotly.express as px

In [4]:
# usually: from dash import Dash
# but for jupyter notebooks use the following
from jupyter_dash import JupyterDash

In [5]:
from dash import (
    html,  # html elements
    dcc,  # dash core (JS-Ui) components
    callback,  # callback on events, e.g. clicks
    Output,  # output elements for callbacks
    Input,  # input elements for callbacks
)

In [6]:
# read example-data
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv"
)
df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
3308,Zimbabwe,Africa,1987,62.351,9216418,706.157306
3309,Zimbabwe,Africa,1992,60.377,10704340,693.420786
3310,Zimbabwe,Africa,1997,46.809,11404948,792.449960
3311,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [7]:
# usually: app = Dash(__name__)
# generate new app instance
app = JupyterDash(__name__)

In [11]:
# create layout by
app.layout = html.Div(
    [
        html.H1(
            children="Simple Dash App", style={"textAlign": "center"}, id="h1_title"
        ),
        dcc.Dropdown(
            df.country.unique(), value="Germany", id="dropdown_selection"
        ),  # https://dash.plotly.com/dash-core-components/dropdown
        dcc.Graph(
            id="graph_content"
        ),  # https://dash.plotly.com/dash-core-components/graph
    ]
)

In [12]:
@callback(Output("graph_content", "figure"), Input("dropdown_selection", "value"))
def update_graph(value):
    dff = df[df.country == value]
    return px.line(dff, x="year", y="pop")

In [13]:
# usually: app.run(debug=True, port=4711, host='0.0.0.0')
app.run_server(mode="inline", debug=True)

Dash is running on http://127.0.0.1:8050/



## Example: Creating a Dash Dashboard with Bootstrap

In [14]:
# bootstrap UI elements
src = "https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/explorer/"
width = 1920
height = 1080
zoom_level = 0.5  # Adjust this value to scale the website

style = f"transform: scale({zoom_level}); transform-origin: top left;"

site = (
    f'<iframe src="{src}" width="{width}" height="{height}" style="{style}"></iframe>'
)
IPython.display.HTML(site)

/home/stefan/private/HSKA/plotly_presentation/venv/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning:

Consider using IPython.display.IFrame instead



In [21]:
import dash_bootstrap_components as dbc

In [22]:
app = JupyterDash(
    __name__, external_stylesheets=[dbc.themes.BOOTSTRAP]
)  # more themes: https://dash-bootstrap-components.opensource.faculty.ai/docs/themes/explorer/
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv"
)

In [23]:
# use bootstrap elements
initial_country = "Germany"
app.layout = html.Div(
    [
        html.H1(
            children="Dash Bootstrap App", style={"textAlign": "center"}, id="h1_title"
        ),
        dcc.Dropdown(
            df.country.unique(), value=initial_country, id="dropdown_selection"
        ),
        dcc.Graph(id="graph_content"),
        dbc.Tooltip(
            "Select a Country", target="dropdown_selection", id="tt_dropdown"
        ),  # reference target by ID
        dbc.Tooltip(
            f"Population growth for {initial_country}", target="h1_title", id="tt_graph"
        ),
    ]
)


@callback(
    [
        Output("graph_content", "figure"),  # Multiple Outputs are now a List
        Output("tt_graph", "children"),
    ],
    Input("dropdown_selection", "value"),
)
def update_graph(country_selection):  # change input variable to useful name
    dff = df[df.country == country_selection]
    return (
        px.line(dff, x="year", y="pop"),
        f"Population growth for {country_selection}",
    )  # return a n-tuple to n outputs


app.run_server(mode="inline", debug=True)

Dash is running on http://127.0.0.1:8050/



In [24]:
from dash import dash_table

df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv"
)

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App layout
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                html.Div(
                    "App with Data, Graph, and Controls",
                    className="text-primary text-center fs-3",
                )
            ]
        ),
        dbc.Row(
            [
                dbc.RadioItems(
                    options=[
                        {"label": x, "value": x}
                        for x in ["pop", "lifeExp", "gdpPercap"]
                    ],
                    value="lifeExp",
                    inline=True,
                    id="radio-buttons-final",
                )
            ]
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dash_table.DataTable(
                            data=df.to_dict("records"),
                            page_size=12,
                            style_table={"overflowX": "auto"},
                        )
                    ],
                    width=6,
                ),
                dbc.Col([dcc.Graph(figure={}, id="my-first-graph-final")], width=6),
            ]
        ),
    ],
    fluid=True,
)


# Add controls to build the interaction
@callback(
    Output(component_id="my-first-graph-final", component_property="figure"),
    Input(component_id="radio-buttons-final", component_property="value"),
)
def update_graph(col_chosen):
    fig = px.histogram(df, x="continent", y=col_chosen, histfunc="avg")
    return fig


app.run_server(mode="inline", debug=True)

Dash is running on http://127.0.0.1:8050/



## Pitfalls
### Multiple Outputs
Multiple Outputs are a List. Every Output can only be used **ONCE**. If you want to use the same Output in multiple callbacks, create a 'master' callback for all Inputs and distinguish the inputs by the `Input`-parameter.
You can use `dash.no_update` to prevent an Output from updating.
```python
@callback(
    [
        Output(component_id='output1', component_property='children'),
        Output(component_id='output2', component_property='children')
    ],
    [
        Input(component_id='button1', component_property='n_clicks'),
        Input(component_id='button1', component_property='n_clicks')
    ]
)
def buttons(btn1_clicks, btn2_clicks):
    if btn1_clicks is None and btn2_clicks is None:
        return dash.no_update, dash.no_update
    elif btn1_clicks is None:
        return dash.no_update, f"clicked {btn2_clicks} times"
    elif btn2_clicks is None:
        return f"clicked {btn1_clicks} times", dash.no_update
    else:
        return f"clicked {btn1_clicks} times", f"clicked {btn2_clicks} times"
```

### State between callbacks
Dash ist stateless, so every user gets an own session. Therefore the Usage of global variables (e.g. the `df` in the examples) is not recommended.
To share data between Callbacks use `dcc.Store()`, a component that stores JSON-serialized data in the users browser-cache.

```python
@callback(
    [
        Output(component_id='output1', component_property='children'),
        Output(component_id='output2', component_property='children'),
        Output(component_id='store1', component_property='data'),
        Output(component_id='store2', component_property='data')
    ],
    [
        Input(component_id='button1', component_property='n_clicks'),
        Input(component_id='button1', component_property='n_clicks')
    ]
    [
        State(component_id='store1', component_property='data'),
        State(component_id='store2', component_property='data')
    ]
)
def buttons(btn1_clicks, btn2_clicks, data1, data2):
    out1, out2, store1, store2 = dash.no_update, dash.no_update, dash.no_update, dash.no_update

    if data1 is None:
        store1 = {'button_clicks': 0}
    if data2 is None:
        store2 = {'button_clicks': 0}

    if btn1_clicks is not None:
        store1['button_clicks'] = btn1_clicks
        out1 = f"clicked {store1['button_clicks']} times"
    if btn2_clicks is not None:
        store2['button_clicks'] = btn2_clicks
        out2 = f"clicked {store2['button_clicks']} times"

    return out1, out2, store1, store2
```

## Alternatives
There are several alternatives to Plotly Dash for creating interactive web applications with Python. Some popular alternatives include:

* https://streamlit.io `Streamlit` is a Python library that allows you to build interactive web applications for data science and machine learning tasks. It focuses on simplicity and ease of use, allowing you to create apps quickly with just a few lines of code.

* https://bokeh.org `Bokeh` is a powerful interactive visualization library for Python. It provides a high-level interface for creating interactive plots, dashboards, and data applications. Bokeh supports a wide range of plot types and can be used in combination with other web frameworks like Flask or Django.

* https://panel.holoviz.org `Panel` is a high-level app and dashboarding solution for Python. It integrates seamlessly with popular plotting libraries like Matplotlib, Bokeh, and Plotly, allowing you to build interactive dashboards with a variety of widgets and layout options.

These alternatives offer different features, levels of complexity, and trade-offs, so you can choose the one that best fits your specific requirements and preferences.

## Conclusion

- Plotly Dash is a powerful Python library for building web-based dashboards and applications.
- It seamlessly integrates with Jupyter Notebooks, allowing you to create interactive visualizations within the familiar Jupyter environment.
- Dash Bootstrap provides additional styling options and components, based on the Bootstrap framework, to enhance the visual appeal of Dash applications.
- By combining Plotly Dash with Dash Bootstrap, you can create interactive and stylish dashboards that effectively communicate your data.